In [1]:
import csv
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [2]:
X = np.loadtxt(open('data/X_train.csv', 'rb'), delimiter=',', skiprows=1)
y = np.loadtxt(open('data/y_train.csv', 'rb'), delimiter=',', skiprows=1)

# Number of datapoints N, input features D
N = len(X)
D = len(X[0])

In [3]:
# entering_host one-hot encoding vector of length 81
entering_host_vectors = np.zeros(shape=(N, 81))

# Map ids to indices
entering_host_ids = []

for i in range(N):
    host_id = X[i][16]
    if host_id not in entering_host_ids:
        entering_host_ids.append(host_id)
    entering_host_vectors[i][entering_host_ids.index(host_id)] = 1

In [4]:
# summoning_host one-hot encoding vector of length 85
summoning_host_vectors = np.zeros(shape=(N, 85))

# Map ids to indices
summoning_host_ids = []

for i in range(N):
    host_id = X[i][25]
    if host_id not in summoning_host_ids:
        summoning_host_ids.append(host_id)
    summoning_host_vectors[i][summoning_host_ids.index(host_id)] = 1

In [5]:
# Delete unnecessary cols (same vals, missing vals, one-hot encoded)
X = np.delete(X, [0, 1, 2, 5, 6, 14, 16, 23, 25], axis=1)
X = np.append(np.append(X, entering_host_vectors, axis=1), summoning_host_vectors, axis=1)

In [6]:
# Split different queue types

X_split = [[] for _ in range(5)]
y_split = [[] for _ in range(5)]

for i in range(N):
    for j in range(2, 7):
        if X[i][j] == 1:
            X_split[j-2].append(X[i])
            y_split[j-2].append(y[i])
            break
            
for i in range(5):
    X_split[i] = np.array(X_split[i])
    y_split[i] = np.array(y_split[i])

In [7]:
# Replace missing values with medians

# q: queue type idx
def replace_missing(q, col_idx):
    X_q = X_split[q]
    col = X_q[:, col_idx]
    med = np.median(col[np.where(col != -1)])
    X_q[:, col_idx] = np.where(col == -1, med, col)
    
for q in range(5):
    replace_missing(q, 7)
    replace_missing(q, 15)

In [8]:
def fit_predict(X_train, y_train, X_test):
    gbr = GradientBoostingRegressor()
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    return y_pred

def fit_predict_rf(X_train, y_train, X_test):
    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    return y_pred

In [23]:
y_test_combined = []
y_pred_combined = []
y_pred_rf_combined = []

for q in range(5):
    X_q, y_q = X_split[q], y_split[q]
    X_train, X_test, y_train, y_test = train_test_split(X_q, y_q, test_size=0.1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test)
    dval = xgb.DMatrix(X_val, label=y_val)
    param = {}
    param['eval_metric'] = ['rmse']
    num_round = 400
    bst = xgb.train(param, dtrain, num_round, evals=[(dval, '')], early_stopping_rounds=10)
    y_pred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)
    
    y_pred_rf = fit_predict_rf(X_train, y_train, X_test)
    
    y_test_combined += list(y_test)
    y_pred_combined += list(y_pred)
    y_pred_rf_combined += list(y_pred_rf)

[08:29:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[0]	-rmse:23.0471
Will train until -rmse hasn't improved in 10 rounds.
[08:29:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[1]	-rmse:20.6673
[08:29:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[2]	-rmse:19.17
[08:29:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[3]	-rmse:18.0553
[08:29:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=6
[4]	-rmse:17.5995
[08:29:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=6
[5]	-rmse:17.2085
[08:29:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=6
[6]	-rmse:17.0104
[08:29:41] src/tree/updater_prune.cc:74

/Users/junekim/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[08:29:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 0 pruned nodes, max_depth=6
[0]	-rmse:91.6848
Will train until -rmse hasn't improved in 10 rounds.
[08:29:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[1]	-rmse:73.1829
[08:29:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=6
[2]	-rmse:61.9121
[08:29:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[3]	-rmse:55.0394
[08:29:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 0 pruned nodes, max_depth=6
[4]	-rmse:50.3504
[08:29:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[5]	-rmse:47.8844
[08:29:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=6
[6]	-rmse:46.4069
[08:29:44] src/tree/updater_prune.cc

[62]	-rmse:39.2928
[08:29:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=6
[63]	-rmse:39.2242
[08:29:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=6
[64]	-rmse:39.2196
[08:29:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[65]	-rmse:39.2527
[08:29:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=6
[66]	-rmse:39.2572
[08:29:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=6
[67]	-rmse:39.2562
[08:29:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[68]	-rmse:39.2532
[08:29:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[69]	-rmse:39.2578
[08:29:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roo

/Users/junekim/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

/Users/junekim/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[08:30:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[0]	-rmse:28.951
Will train until -rmse hasn't improved in 10 rounds.
[08:30:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[1]	-rmse:24.8607
[08:30:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[2]	-rmse:22.2743
[08:30:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[3]	-rmse:20.7679
[08:30:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[4]	-rmse:19.8885
[08:30:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=6
[5]	-rmse:19.4843
[08:30:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[6]	-rmse:19.1568
[08:30:04] src/tree/updater_prun

/Users/junekim/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[08:30:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=6
[0]	-rmse:10.5743
Will train until -rmse hasn't improved in 10 rounds.
[08:30:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=6
[1]	-rmse:10.246
[08:30:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=6
[2]	-rmse:9.94375
[08:30:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=6
[3]	-rmse:9.67959
[08:30:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=6
[4]	-rmse:9.48884
[08:30:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=6
[5]	-rmse:9.31367
[08:30:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=6
[6]	-rmse:9.18284
[08:30:07] src/tree/updater_prune.cc:7

/Users/junekim/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [24]:
mean_squared_error(y_test_combined, y_pred_combined)

1080.117980824731

In [25]:
mean_squared_error(y_test_combined, y_pred_rf_combined)

1155.6552032324576

In [33]:
y_pred_avg = 0.7 * np.array(y_pred_combined) + 0.3 * np.array(y_pred_rf_combined)
mean_squared_error(y_test_combined, y_pred_avg)

1023.4463176505811

In [45]:
# For submission

X_test_real = np.loadtxt(open('data/X_test.csv', 'rb'), delimiter=',', skiprows=1)
N = len(X_test_real)

# entering_host one-hot encoding vector of length 81
entering_host_vectors = np.zeros(shape=(N, 81))

for i in range(len(X_test_real)):
    host_id = X_test_real[i][16]
    if host_id not in entering_host_ids:
        break
    entering_host_vectors[i][entering_host_ids.index(host_id)] = 1

    
# summoning_host one-hot encoding vector of length 85
summoning_host_vectors = np.zeros(shape=(N, 85))

for i in range(len(X_test_real)):
    host_id = X_test_real[i][25]
    if host_id not in summoning_host_ids:
        break
    summoning_host_vectors[i][summoning_host_ids.index(host_id)] = 1
    
    
# Delete unnecessary cols (same vals, missing vals, one-hot encoded)
X_test_real = np.delete(X_test_real, [0, 1, 2, 5, 6, 14, 16, 23, 25], axis=1)
X_test_real = np.append(np.append(X_test_real, entering_host_vectors, axis=1), summoning_host_vectors, axis=1)


# Split different queue types
X_test_split = [[] for _ in range(5)]
X_ids_split = [[] for _ in range(5)]

for i in range(len(X_test_real)):
    for j in range(2, 7):
        if X_test_real[i][j] == 1:
            X_test_split[j-2].append(X_test_real[i])
            X_ids_split[j-2].append(i)
            break
            
for i in range(5):
    X_test_split[i] = np.array(X_test_split[i])

    
# Replace missing values with medians

# q: queue type idx
def replace_missing(q, col_idx):
    X_q = X_test_split[q]
    col = X_q[:, col_idx]
    med = np.median(col[np.where(col != -1)])
    X_q[:, col_idx] = np.where(col == -1, med, col)
    
for q in range(5):
    replace_missing(q, 7)
    replace_missing(q, 15)
    
    
# Train and predict

y_pred_combined = []
y_pred_rf_combined = []

for q in range(5):
    # Train Xgboost
    X_train, y_train = X_split[q], y_split[q]
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)
    X_test = X_test_split[q]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test)
    dval = xgb.DMatrix(X_val, label=y_val)
    param = {}
    param['eval_metric'] = ['rmse']
    num_round = 400
    bst = xgb.train(param, dtrain, num_round, evals=[(dval, '')], early_stopping_rounds=10)
    y_pred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)
    y_pred_combined.append(y_pred)
    
    # Train RF
    y_pred_rf = fit_predict_rf(X_train, y_train, X_test)
    y_pred_rf_combined.append(y_pred_rf)
    
# Sort by id
y_pred_sorted = [0 for _ in range(N)]
y_pred_rf_sorted = [0 for _ in range(N)]

for q in range(5):
    for i in range(len(y_pred_combined[q])):
        y_pred_sorted[X_ids_split[q][i]] = y_pred_combined[q][i]
        y_pred_rf_sorted[X_ids_split[q][i]] = y_pred_rf_combined[q][i]

[08:58:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[0]	-rmse:21.1216
Will train until -rmse hasn't improved in 10 rounds.
[08:58:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=6
[1]	-rmse:19.1877
[08:58:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=6
[2]	-rmse:18.2057
[08:58:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[3]	-rmse:17.6989
[08:58:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[4]	-rmse:17.5555
[08:58:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=6
[5]	-rmse:17.4701
[08:58:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=6
[6]	-rmse:17.4136
[08:58:50] src/tree/updater_prune.cc:

/Users/junekim/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[08:58:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 0 pruned nodes, max_depth=6
[0]	-rmse:92.2947
Will train until -rmse hasn't improved in 10 rounds.
[08:58:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 0 pruned nodes, max_depth=6
[1]	-rmse:74.2809
[08:58:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[2]	-rmse:62.8935
[08:58:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[3]	-rmse:55.99
[08:58:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 0 pruned nodes, max_depth=6
[4]	-rmse:51.6638
[08:58:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[5]	-rmse:48.0609
[08:58:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=6
[6]	-rmse:46.3339
[08:58:53] src/tree/updater_prune.cc:

[08:59:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=6
[63]	-rmse:39.5755
[08:59:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=6
[64]	-rmse:39.5338
[08:59:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=6
[65]	-rmse:39.536
[08:59:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[66]	-rmse:39.5126
[08:59:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[67]	-rmse:39.4924
[08:59:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[68]	-rmse:39.4818
[08:59:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 0 pruned nodes, max_depth=6
[69]	-rmse:39.4802
[08:59:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra node

/Users/junekim/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

/Users/junekim/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[08:59:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[0]	-rmse:26.2117
Will train until -rmse hasn't improved in 10 rounds.
[08:59:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[1]	-rmse:22.4172
[08:59:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[2]	-rmse:20.21
[08:59:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[3]	-rmse:18.893
[08:59:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[4]	-rmse:18.117
[08:59:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=6
[5]	-rmse:17.805
[08:59:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[6]	-rmse:17.5113
[08:59:23] src/tree/updater_prune.c

/Users/junekim/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[08:59:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=6
[0]	-rmse:3.50647
Will train until -rmse hasn't improved in 10 rounds.
[08:59:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=6
[1]	-rmse:3.45203
[08:59:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=6
[2]	-rmse:3.41686
[08:59:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=6
[3]	-rmse:3.42378
[08:59:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=6
[4]	-rmse:3.44935
[08:59:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[5]	-rmse:3.45006
[08:59:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=6
[6]	-rmse:3.50393
[08:59:25] src/tree/updater_prune.cc:

/Users/junekim/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [48]:
y_pred_avg = (0.3 * np.array(y_pred_1) + 0.3 * np.array(y_pred_2) 
              + 0.3 * np.array(y_pred_3) + 0.1 * np.array(y_pred_rf_sorted))

print(y_pred_avg)

[35.042826   20.28346359 47.70979614 ... 19.98409904 18.99769806
 19.70409904]


In [49]:
with open('submissions/sub10.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerow(['id', 'actual_wait div 60000'])
    for i in range(len(y_pred_sorted)):
        writer.writerow([i, y_pred_avg[i]])

In [40]:
y_pred_1 = y_pred_sorted.copy()

In [43]:
y_pred_2 = y_pred_sorted.copy()

In [46]:
y_pred_3 = y_pred_sorted.copy()

In [47]:
print(y_pred_1[:20])
print(y_pred_2[:20])
print(y_pred_3[:20])

[42.470417, 18.702324, 45.77598, 2.7884226, 1.6310292, 21.689177, 1.3452116, 2.4738905, 41.271805, 19.07518, 2.2300336, 19.07518, 45.44544, 2.7884226, 19.07518, 29.559347, 2.915995, 28.28845, 1.6832705, 14.958442]
[25.649424, 20.334211, 46.528076, 2.8623097, 1.9949447, 22.376505, 1.7673854, 2.3076057, 36.396847, 11.017347, 2.3153036, 11.017347, 49.66821, 2.766684, 11.017347, 26.582817, 2.7093341, 26.74, 4.0804358, 26.060625]
[37.62291, 20.675007, 49.39526, 2.976402, 1.0305676, 22.94888, 1.0305676, 2.1439617, 44.59673, 16.897446, 2.4424183, 16.897446, 44.33706, 2.976402, 16.897446, 28.443594, 2.4424183, 25.34985, 1.0305676, 22.005596]


In [35]:
y_pred_rf_sorted

[47.4,
 19.5,
 52.3,
 6.1,
 3.0,
 24.1,
 1.7,
 3.7,
 37.2,
 5.814285714285715,
 3.1,
 5.814285714285715,
 66.09580952380954,
 5.7,
 5.814285714285715,
 16.1,
 2.6,
 23.8,
 1.8,
 24.9,
 67.7,
 66.09580952380954,
 23.7,
 3.5,
 1.4,
 12.3,
 22.55,
 26.4,
 1.6,
 22.1,
 2.1,
 7.4,
 3.2,
 61.1,
 1.3,
 1.8,
 2.4,
 1.6,
 24.8,
 1.9,
 23.9,
 1.6,
 1.0,
 3.6,
 28.2,
 1.9,
 0.8,
 0.3,
 43.0,
 1.6,
 38.6,
 31.8,
 22.6,
 2.4,
 0.9,
 8.4,
 0.3,
 16.5,
 16.8,
 1.2,
 21.1,
 12.7,
 15.1,
 29.2,
 20.1,
 14.1,
 66.0,
 2.4,
 79.7,
 65.9,
 2.9,
 41.8,
 61.7,
 81.43333333333334,
 63.6,
 76.0,
 66.64,
 78.1,
 81.55,
 74.0,
 75.71666666666667,
 65.8,
 66.5,
 60.6,
 61.9,
 3.0,
 2.2,
 56.1,
 2.7,
 60.6,
 58.4,
 3.5,
 2.5,
 2.6,
 1.6,
 41.542857142857144,
 3.5,
 55.7,
 53.5,
 0.0,
 83.05,
 3.4,
 99.0,
 99.8,
 2.7,
 89.63333333333334,
 30.206078110431925,
 68.5,
 68.0,
 60.6,
 36.3,
 55.19285714285714,
 59.5,
 3.2,
 45.55,
 36.8,
 68.935,
 3.3,
 57.9,
 78.4,
 44.55,
 48.53333333333333,
 2.5,
 54.4,
 71.8,
 73.9,